In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import networkx as nx
import collections
from typing import Sequence, Tuple, List
from tracker import extractor, utils, metrics, visuals

In [2]:
def cartesian(position: np.ndarray, order: List[str]) -> np.ndarray:
    phi, r, z = [position[order.index(s)] for s in ["phi", "r", "z"]]
    return np.array([(np.cos(phi) * r), (np.sin(phi) * r), z])

def groupby(array: np.ndarray) -> List[List[float]]:
    D = dict((value, index) for (index, value) in enumerate(np.unique(array)))
    L = [[] for _ in range(len(D))]
    for index, value in enumerate(array):
        L[D[value]].append(index)
    return L

def intersection(
        p0 : np.ndarray,  # The center of the first circle.
        r0 : float,       # The radius of the first circle.
        p1 : np.ndarray,  # The center of the second circle.
        r1 : float,       # The radius of the second circle.
        ) -> Tuple[np.ndarray, np.ndarray]:  # Two points of intersection.
    d  = np.linalg.norm(p0 - p1)
    a  = (r0 * r0 - r1 * r1 + d * d) / (2 * d)
    h  = np.sqrt(r0 * r0 - a * a)
    p2 = (p1 - p0) * (a / d) + p0
    x3 = p2[0] + h * (p1[1] - p0[1]) / d
    y3 = p2[1] - h * (p1[0] - p0[0]) / d
    x4 = p2[0] - h * (p1[1] - p0[1]) / d
    y4 = p2[1] + h * (p1[0] - p0[0]) / d
    return np.array([x3, y3]), np.array([x4, y4])

def phi_bounds_from_circle(xyz: np.ndarray, radius: float) -> Tuple[float, float]:
    center = xyz[:2] / 2
    b0, b1 = intersection(center, np.linalg.norm(center), np.array([0, 0]), radius)
    return  np.arctan2(b1[1], b1[0]), np.arctan2(b0[1], b0[0])

def phi_bounds_from_lines(super_xyz: np.ndarray, sub_xyz: np.ndarray, radius: float) -> Tuple[float, float]:
    x0, y0 = super_xyz[:2]
    x1, y1 = sub_xyz[:2]
    b0 = np.arctan2(y1, x1)
    return None

def phi_lies_within_bounds(phi: float, bounds: Tuple[float, float]) -> bool:
    phi = phi % (2 * np.pi)
    b1  = bounds[0] % (2 * np.pi)
    b2  = bounds[1] % (2 * np.pi)
    return (b1 <= phi or phi <= b2) if (b1 > b2) else (b1 <= phi and phi <= b2)

def tighten_bounds(bounds1: Tuple[float, float], bounds2: Tuple[float, float]) -> Tuple[float, float]:
    b1 = bounds1[0] if phi_lies_within_bounds(bounds2[0], bounds1) else bounds2[0]
    b2 = bounds1[1] if phi_lies_within_bounds(bounds2[1], bounds1) else bounds2[1]
    return b1, b2

In [3]:
%%time
frame  = pd.read_csv("data/sets/RAMP-10N-25T-3600E-235R.gz")
frame  = utils.remove_padding(utils.remove_noise(frame))
frame  = frame.groupby(["event_id", "cluster_id"]).filter(lambda group: len(group) == 9)
events = utils.list_of_groups(frame, "event_id")
events = sorted(events, key=(lambda event: len(event)))

Wall time: 4.87 s


In [ ]:
order     = ["phi", "r", "z"]
number    = 400
print("Hits:   {}".format(metrics.number_of_hits(events[number])))
print("Tracks: {}".format(metrics.number_of_tracks(events[number])))
positions = extractor.extract_input(events[number], order)
tracks    = extractor.extract_output(events[number], order, categorical=False)

In [ ]:
G = nx.DiGraph()
for idx, position in enumerate(positions):
    G.add_node(
        n=idx,
        cartesian=cartesian(position, order)[:2],
        phi=position[order.index("phi")],
        r=position[order.index("r")],
        z=position[order.index("z")]
    )
nodes    = G.nodes(data=True)
radiuses = groupby(positions[:, order.index("r")])
print(radiuses)

In [ ]:
def connect(
        nodes_index    : int, 
        radiuses_index : int, 
        bounds         : Tuple[float, float]
        ) -> List[int]:
    cartesian  = nodes[nodes_index][1]["cartesian"]  # 1D array
    candidates = radiuses[radiuses_index]  # List[int]
    radius     = nodes[candidates[0]][1]["r"]
    
    circle_bounds = phi_bounds_from_circle(cartesian, radius)
    line_bounds   = phi_bounds_from_line()
    bounds = circle_bounds if bounds is None else bounds
    bounds = tighten_bounds(bounds, circle_bounds)
    bounds = tighten_bounds(bounds, line_bounds)
    
    accepted   = []
    for i in candidates:
        node = nodes[i][1]
        phi  = node["phi"]
        if phi_lies_within_bounds(phi, bounds):
            if radiuses_index <= 0:
                accepted.append((nodes_index, i))
            else:
                next_branches = connect(i, radiuses_index - 1, bounds)
                if next_branches:
                    accepted += next_branches
                    accepted.append((nodes_index, i))
    return accepted

for i in radiuses[-1]:
    print(i)
    G.add_edges_from(connect(i, len(radiuses) - 2, None))

In [ ]:
nx.draw_networkx(G, pos=nx.get_node_attributes(G, "cartesian"), node_size=20)